# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine

# import nltlk (Natural Language Toolkit)
import nltk
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

# import scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report
from sklearn.multioutput import MultiOutputClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# import picke to save a trained model
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
X = df['message'].values
Y = df.iloc[:,4:]
category_names = Y.columns.tolist()

In [3]:
X.dtype

dtype('O')

In [15]:
pd.set_option('display.max_columns', 500)
df.head()
df["num_words_original"]= df.message.str.split(" ").apply(len)
df["num_words_"]= df.message.str.split(" ")
# df.new.head()
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products  \
0        0      0            0             0                 0   
1        0      0            1             0                 0   
2        0      0            0             0                 0   
3        1      0            1             0                 1   
4        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  water  food  shelter  \
0                  0         0         0            0      0     0        0   
1                  0         0         0            0      0     0        0   
2                  0         0         0            0      0     0        0   
3                  0         0         0            0      0     0        0   
4                  0         0         0            0      0     0        0   

   clothing  money  missing_people  refugees  death  other_aid  \
0         0      0               0         0      0          0   
1         0      0               0         0      0          1   
2         0      0               0         0      0          0   
3         0      0               0         0      0          1   
4         0      0               0         0      0          0   

   infrastructure_related  transport  buildings  electricity  tools  \
0                       0          0          0            0      0   
1                       0          0          0            0      0   
2                       0          0          0            0      0   
3                       1          0          1            0      0   
4                       0          0          0            0      0   

   hospitals  shops  aid_centers  other_infrastructure  weather_related  \
0          0      0            0                     0                0   
1          0      0            0                     0                1   
2          0      0            0                     0                0   
3          1      0            0                     0                0   
4          0      0            0                     0                0   

   floods  storm  fire  earthquake  cold  other_weather  direct_report  \
0       0      0     0           0     0              0              0   
1       0      1     0           0     0              0              0   
2       0      0     0           0     0              0              0   
3       0      0     0           0     0              0              0   
4       0      0     0           0     0              0              0   

   num_words_original                                         num_words_  
0                  13  [Weather, update, -, a, cold, front, from, Cub...  
1                   9  [Is, the, Hurricane, over, or, is, it, not, over]  
2                   6             [Looking, for, someone, but, no, name]  
3                  13  [UN, reports, Leogane, 80-90, destroyed., Only...  
4                  12  [says:, west, side, of, Haiti,, rest, of, the,...

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    '''
    Function splits text into separate words and gets a word lowercased and removes whitespaces at the ends of a word. The funtions also cleans irrelevant stopwords.
    
    Input:
    1. text: text message
    
    Output:
    1. Clean_tokens : list of tokenized clean words
    '''
    # Get rid of other sepcial characters   
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Tokenize
    tokens = word_tokenize(text)
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok, pos='v').lower().strip()
        clean_tokens.append(clean_tok)
    
    # Remove stop words    
    stopwords = nltk.corpus.stopwords.words('english')
    clean_tokens = [token for token in clean_tokens if token not in stopwords]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
class TokenCounter(BaseEstimator, TransformerMixin):
    def count_tokens(self, text):
        tokens = nltk.word_tokenize(text)
        number_of_tokens = len(tokens)
        return  number_of_tokens
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.count_tokens)
        return pd.DataFrame(X_tagged)

In [18]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('counttk', TokenCounter())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state =42)

print(type(y_train))
y_train.head()

<class 'pandas.core.frame.DataFrame'>


related  request  offer  aid_related  medical_help  medical_products  \
5326         1        1      0            1             0                 1   
10971        1        0      0            1             0                 0   
22543        0        0      0            0             0                 0   
9989         1        0      0            1             0                 0   
15105        1        0      0            1             0                 0   

       search_and_rescue  security  military  child_alone  water  food  \
5326                   0         0         0            0      0     1   
10971                  1         0         0            0      0     0   
22543                  0         0         0            0      0     0   
9989                   0         0         0            0      0     0   
15105                  0         0         0            0      0     0   

       shelter  clothing  money  missing_people  refugees  death  other_aid  \
5326         0         0      0               0         0      0          1   
10971        0         0      0               0         0      0          1   
22543        0         0      0               0         0      0          0   
9989         0         0      0               0         0      0          1   
15105        0         0      0               0         0      1          0   

       infrastructure_related  transport  buildings  electricity  tools  \
5326                        0          0          0            0      0   
10971                       1          1          0            0      0   
22543                       0          0          0            0      0   
9989                        0          0          0            0      0   
15105                       0          1          0            0      0   

       hospitals  shops  aid_centers  other_infrastructure  weather_related  \
5326           0      0            0                     0                1   
10971          0      0            1                     0                1   
22543          0      0            0                     0                0   
9989           0      0            0                     0                1   
15105          0      0            0                     0                1   

       floods  storm  fire  earthquake  cold  other_weather  direct_report  
5326        0      0     1           0     0              0              1  
10971       1      0     0           0     0              0              0  
22543       0      0     0           0     0              0              0  
9989        0      0     0           1     0              0              0  
15105       0      0     0           1     0              0              0

In [20]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
def display_results(y_test, y_pred, category_names):
    for i in range(0, len(category_names)):
        print(category_names[i])
        print(classification_report(y_test[category_names[i]], y_pred[:, i]))
        print("f1_score of {} is {:.2f}".format(category_names[i], f1_score(y_test[category_names[i]], y_pred[:, i])))
        print("**"*30)

In [ ]:
display_results(y_test, y_pred, category_names)

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('counttk', TokenCounter())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators

In [14]:
parameters = {
    'features__text_pipeline__vect__max_df': (0.5,1.0),
    'clf__estimator__n_estimators': [50,100]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=10)
cv.fit(X_train, y_train)
cv.best_params_

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=0.5 
[CV]  clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=0.5, score=0.2525175961017867, total= 2.2min
[CV] clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=0.5 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=0.5, score=0.25695722793719544, total= 2.2min
[CV] clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=1.0 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  5.0min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=1.0, score=0.2564158094206822, total= 2.2min
[CV] clf__estimator__n_estimators=50, features__text_pipeline__vect__max_df=1.0 


...

[CV]  clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0, score=0.2563075257173795, total= 4.1min
[CV] clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0 


[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed: 23.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0, score=0.25284244721169463, total= 4.0min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 28.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 28.1min finished


{'clf__estimator__n_estimators': 50,
 'features__text_pipeline__vect__max_df': 0.5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
pipeline.set_params(**cv.best_params_)
pipeline.fit(X_train, y_train)
y_pred_1 = pipeline.predict(X_test)
display_results(y_test, y_pred_1, category_names)

related
             precision    recall  f1-score   support

          0       0.70      0.38      0.49      1844
          1       0.83      0.95      0.89      6072

avg / total       0.80      0.82      0.80      7916

f1_score of related is 0.89
************************************************************
request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      6560
          1       0.84      0.49      0.62      1356

avg / total       0.89      0.90      0.89      7916

f1_score of request is 0.62
************************************************************
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7879
          1       1.00      0.03      0.05        37

avg / total       1.00      1.00      0.99      7916

f1_score of offer is 0.05
************************************************************
aid_related
             precision    recall  f1-score   support

  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Already TokenCounter class was used, so skip this.

### 9. Export your model as a pickle file

In [22]:
pickle.dump(pipeline, open("classifier.pkl", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.